In [1]:
%pip install snowflake-connector-python pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
from snowflake.connector import connect
from snowflake.connector.pandas_tools import write_pandas

df = pd.read_csv("train_u6lujuX_CVtuZ9i.csv")

conn = connect(
    user='OJASDJOSHI',
    password='VWj6SQdd4hPmY9Q',
    account='ZWUTDGD-TC94222',
    warehouse='VISTORA_WH',
    database='VISTORA_DB',
    schema='VISTORA_PROJECT'
)
# Snowflake automatically detects the data type of all the columns
write_pandas(conn, df, table_name='loan_train', auto_create_table=True, overwrite=True)


(True,
 1,
 614,
 [('ehdpubwwln/file0.txt', 'LOADED', 614, 614, 1, 0, None, None, None, None)])

In [ ]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

#Handling missing values
df['Gender'] = df['Gender'].fillna(df['Gender'].mode()[0])
df['Loan_Amount_Term'] = df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].mode()[0])
df['LoanAmount'] = df['LoanAmount'].fillna(df['LoanAmount'].median())
df['Credit_History'] = df['Credit_History'].fillna(df['Credit_History'].mode()[0])
df['Self_Employed'] = df['Self_Employed'].fillna(df['Self_Employed'].mode()[0])
df['Dependents'] = df['Dependents'].fillna(df['Dependents'].mode()[0])
df['Married'] = df['Married'].fillna(df['Married'].mode()[0])
df['Education'] = df['Education'].fillna(df['Education'].mode()[0])
df['Property_Area'] = df['Property_Area'].fillna(df['Property_Area'].mode()[0])
df['Loan_Status'] = df['Loan_Status'].fillna(df['Loan_Status'].mode()[0])

# Encode target variable
df['Loan_Status'] = df['Loan_Status'].map({'Y': 1, 'N': 0})

# Create new features
df['Income_Loan_Ratio'] = df['ApplicantIncome'] / df['LoanAmount']
df['Total_Income'] = df['ApplicantIncome'] + df['CoapplicantIncome']
df['Income_Bin'] = pd.qcut(df['ApplicantIncome'], 4, labels=False) #qcut ensures equal data points in each bin
df['EMI'] = df['LoanAmount'] / df['Loan_Amount_Term']

le = LabelEncoder()
df['Dependents'] = le.fit_transform(df['Dependents'].astype(str))  

# One hot encode categorical columns
categorical_cols = ['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

df = df.drop(columns=['Loan_ID'], errors='ignore')

df.head()


,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Income_Loan_Ratio,Total_Income,Income_Bin,EMI,Gender_Male,Married_Yes,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban
0,0,5849,0.0,128.0,360.0,1.0,1,45.695312,5849.0,3,0.355556,True,False,False,False,False,True
1,1,4583,1508.0,128.0,360.0,1.0,0,35.804688,6091.0,2,0.355556,True,True,False,False,False,False
2,0,3000,0.0,66.0,360.0,1.0,1,45.454545,3000.0,1,0.183333,True,True,False,True,False,True
3,0,2583,2358.0,120.0,360.0,1.0,1,21.525000,4941.0,0,0.333333,True,True,True,False,False,True
4,0,6000,0.0,141.0,360.0,1.0,1,42.553191,6000.0,3,0.391667,True,False,False,False,False,True


In [ ]:
df.shape

(614, 17)

In [ ]:
from snowflake.connector.pandas_tools import write_pandas

# Clean column names for Snowflake compatibility
df.columns = [col.strip().lower().replace(" ", "_").replace('"', '') for col in df.columns]

# Upload the engineered features to Snowflake feature store
success, nchunks, nrows, _ = write_pandas(
    conn,                      
    df,                        
    table_name='loan_feature_store',  
    auto_create_table=True,          
    overwrite=True                   
)

print(f"Upload success: {success}, Rows uploaded: {nrows}")


Upload success: True, Rows uploaded: 614


In [ ]:
import pandas as pd

# Query to read from the feature store table
conn = connect(
    user='OJASDJOSHI',
    password='VWj6SQdd4hPmY9Q',
    account='ZWUTDGD-TC94222',
    warehouse='VISTORA_WH',
    database='VISTORA_DB',
    schema='VISTORA_PROJECT'
)
query = 'SELECT * FROM "loan_feature_store";'
df_feat = pd.read_sql(query, conn)

df_feat.head()


C:\Users\ojasd\AppData\Local\Temp\ipykernel_30844\2766188260.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_feat = pd.read_sql(query, conn)


,dependents,applicantincome,coapplicantincome,loanamount,loan_amount_term,credit_history,loan_status,income_loan_ratio,total_income,income_bin,emi,gender_male,married_yes,education_not_graduate,self_employed_yes,property_area_semiurban,property_area_urban
0,0,5849,0.0,128.0,360.0,1.0,1,45.695312,5849.0,3,0.355556,True,False,False,False,False,True
1,1,4583,1508.0,128.0,360.0,1.0,0,35.804688,6091.0,2,0.355556,True,True,False,False,False,False
2,0,3000,0.0,66.0,360.0,1.0,1,45.454545,3000.0,1,0.183333,True,True,False,True,False,True
3,0,2583,2358.0,120.0,360.0,1.0,1,21.525000,4941.0,0,0.333333,True,True,True,False,False,True
4,0,6000,0.0,141.0,360.0,1.0,1,42.553191,6000.0,3,0.391667,True,False,False,False,False,True


In [ ]:
# Drop redundant feature and target to form X
X = df_feat.drop(columns=['loan_status','applicantincome', 'coapplicantincome', 'income_bin', 'loan_amount_term'])

# Target column 
y = df_feat['loan_status']

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42)

model.fit(X,y)
from sklearn.model_selection import cross_val_score

# cross validation
cv_scores = cross_val_score(model, X, y, cv=25, scoring='accuracy')

# Print accuracy
print("Cross-validation accuracies for each fold:", cv_scores)
print(f"Mean cross-validated accuracy: {cv_scores.mean() * 100:.2f}%")

Cross-validation accuracies for each fold: [0.76       0.76       0.88       0.84       0.76       0.76
 0.72       0.8        0.76       0.76       0.6        0.8
 0.84       0.8        0.75       0.91666667 0.875      0.83333333
 0.79166667 0.875      0.70833333 0.75       0.91666667 0.79166667
 0.79166667]
Mean cross-validated accuracy: 79.36%


In [123]:
df_test=pd.read_csv("test_Y3wMUE5_7gLdaTN.csv")
df_test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [ ]:
#Feature engineering on testing datset

df_test['Loan_Amount_Term'] = df_test['Loan_Amount_Term'].fillna(df_test['Loan_Amount_Term'].mode()[0])
df_test['Gender'] = df_test['Gender'].fillna(df_test['Gender'].mode()[0])
df_test['LoanAmount'] = df_test['LoanAmount'].fillna(df_test['LoanAmount'].median())
df_test['Credit_History'] = df_test['Credit_History'].fillna(df_test['Credit_History'].mode()[0])
df_test['Self_Employed'] = df_test['Self_Employed'].fillna(df_test['Self_Employed'].mode()[0])
df_test['Dependents'] = df_test['Dependents'].fillna(df_test['Dependents'].mode()[0])
df_test['Married'] = df_test['Married'].fillna(df_test['Married'].mode()[0])
df_test['Education'] = df_test['Education'].fillna(df_test['Education'].mode()[0])
df_test['Property_Area'] = df_test['Property_Area'].fillna(df_test['Property_Area'].mode()[0])

# Create new engineered features

# Income-to-loan ratio
df_test['Income_Loan_Ratio'] = df_test['ApplicantIncome'] / df_test['LoanAmount']

# Total income 
df_test['Total_Income'] = df_test['ApplicantIncome'] + df_test['CoapplicantIncome']

# Bin applicant income into 4 quantiles
df_test['Income_Bin'] = pd.qcut(df_test['ApplicantIncome'], 4, labels=False)

df_test['EMI'] = df_test['LoanAmount'] / df_test['Loan_Amount_Term']

le = LabelEncoder()
df_test['Dependents'] = le.fit_transform(df_test['Dependents'].astype(str))  # '3+' will be handled

# Step 4: One-hot encode categorical columns
categorical_cols = ['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area']
df_test = pd.get_dummies(df_test, columns=categorical_cols, drop_first=True)

# Step 5: Drop identifier column if exists
df_test = df_test.drop(columns=['Loan_ID', 'ApplicantIncome', 'CoapplicantIncome', 'Income_Bin', 'Loan_Amount_Term'], errors='ignore')

# Final preview
df_test.head()


,Dependents,LoanAmount,Credit_History,Income_Loan_Ratio,Total_Income,EMI,Gender_Male,Married_Yes,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban
0,0,110.0,1.0,52.000000,5720,0.305556,True,True,False,False,False,True
1,1,126.0,1.0,24.412698,4576,0.350000,True,True,False,False,False,True
2,2,208.0,1.0,24.038462,6800,0.577778,True,True,False,False,False,True
3,2,100.0,1.0,23.400000,4886,0.277778,True,True,False,False,False,True
4,0,78.0,1.0,42.000000,3276,0.216667,True,False,True,False,False,True


In [ ]:
df_test.columns = [col.strip().lower().replace(" ", "_").replace('"', '') for col in df_test.columns]
# Predict loan status for the test dataset
predictions = model.predict(df_test)

# Add predictions to the test DataFrame
df_test['Loan_Status_Predicted'] = predictions
df_test[['Loan_Status_Predicted']].head()


,Loan_Status_Predicted
0,1
1,1
2,1
3,1
4,0


In [133]:
success, nchunks, nrows, _ = write_pandas(
    conn,                      
    df_test,                        
    table_name='tesing_data_predictions',  
    auto_create_table=True,          
    overwrite=True                   
)

print(f"Upload success: {success}, Rows uploaded: {nrows}")

Upload success: True, Rows uploaded: 367
